In [2]:
import tensorflow as tf
from tensorflow import keras

# 옵티마이저
자세한 수식은 책을 참고<br>
- 모멘텀 최적화
- 네스테로프 가속 경사
- AdaGrad
- RMSProp
- Adam / AdaMax / Nadam

In [1]:
# 모멘텀 최적화 :  그레디언트를 속도가 아닌 가속도 개념으로
optimizer = keras.optimizers.SGD(lr=0.001,momentum=0.9)

# 네스테로프 가속 경사 : 현재 위치 theta가 아닌 모멘텀 방향으로 조금 앞선 theta+beta*m에서의 비용함수의 그래디언트 계산
optimizer = keras.optimizers.SGD(lr=0.001,momentum=0.9, nesterov=True)

# AdaGrad :  전역 최적점 방향으로 곧장 향하지 않고 가장 가파른 경사를 따라 빠르게 내려가다가 완만한 경사에서는 느리게 이동

# RMSProp : AdaGrad가 너무 빨리 느러져 전역 최적점에 수렴하지 못하는 문제 해결
optimizer = keras.optimizers.RMSprop(lr=0.001,rho=0.9)

# Adam : 모멘텀 최적화와 RMSProp의 아이디어를 합친 것
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

## 학습률 스케줄링
- 거듭제곱 기반 스케줄링
- 지수 기반 스케줄링
- 구간별 고정 스케줄링
- 성능 기반 스케줄링
- 1사이클 스케줄링

In [ ]:
# 거듭제곱 기반 스케줄링 : s는 학습률을 반으로 나누기 위해 수행할 스텝의 수의 역수
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)

# 지수 기반 스케줄링
def exponential_decay_fn(epoch):
  return 0.01*0.1**(epoch/20)

# eta와 s를 하드코딩하고 싶지 않으면 이 변수를 설정한 클로저를 반환하는 함수를 만들 수 있다.
def exponential_decay(lr0,s):
  def exponential_decay_fn(epoch):
    return lr0*0.1**(epoch/s)
  return exponential_decay_fn
exponential_decay_fn = exponential_decay(lr0=0.01,s=20)

# LearningRateScheduler 콜백 : 에포크 시작마다 옵티마이저의 학습률 속성을 업데이트
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train,y_train,validation_data = (X_val,y_val),epochs=10, callbacks=[lr_scheduler])

In [ ]:
# 구간별 고정 스케줄링
def piecewise_contant_fn(epoch):
  if epoch<5:
    return 0.01
  elif epoch<15:
    return 0.005
  else:
    return 0.001

# 성능기반 스케줄링 : ReduceLROnPlateau 콜백을 사용
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
# 최상의 검증 손실이 다섯 번의 연속적인 에포크 동안 향상되지 않을 때마다 학습률에 0.5를 곱한다.

# 에포크가 아닌 매 스텝마다 학습률 업데이트하기 : 지수 기반 스케줄링
s = 20*len(X_train)//32  # 20번 에포크에 담긴 전체 스텝 수(배치 크기 = 32)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01,s,0.1)
optimizer = keras.optimizers.SGD(learning_rate)

# 규제(과대적합 피하기)
$l_1$,$l_2$규제, 드롭아웃, 맥스-노름 규제

## $l_1$과 $l_2$ 규제
신경망의 연결 가중치를 제한하기 위해 $l_2$ 규제 사용<br>
희소 모델을 만들기 위해 $l_1$규제 사용

In [ ]:
# 케라스 층의 연결 가중치에 규제 강도 0.01을 사용하여 l2 구제를 적용하기
layer = keras.layers.Dense(100,activation='elu',
                           kernel_initializer='he_normal',
                           kernel_regularizer = keras.regularizers.l2(0.01))
# l1이 필요하면 l2 -> l1
# 둘다 필요하면 keras.regulizers.l1_l2()

In [2]:
# 모든 층에 동일한 규제를 적용하고 같은 활성화 함수, 동일한 초기화 전략을 사용하기 때문에 매개변수 값을 반복하는 경우가 많다.
# 가독성이 떨어질 수 있기 때문에 반복문을 사용하도록 코드를 리팩터링(refactoring)할 수 있다.
# functools.partial() 함수를 사용하여 기본 매개변수 값을 사용하여 함수 호출을 감싼다.

from functools import partial
RegularizedDense = partial(keras.layers.Dense,
                           activation='elu',
                           kernel_initializer='he_normal',
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
                                 keras.layers.Fatten(input_shape=[28,28]),
                                 RegularizedDense(300),
                                 RegularizedDense(100),
                                 RegularizedDense(10,activation='softmax',kernel_initializer='glorot_uniform')
])

## 드롭 아웃(dropout)
매 훈련 스텝에서 입출력층을 제외한 각 뉴런은 임시적으로 드롭아웃될 확률 p를 가진다. 즉 이번 훈련 스텝에서는 완전히 무시되지만 다음 스텝에서는 활성화될 수 있다.

하이퍼파라미터 p를 드롭아웃 비율이라고 하며 보통 10~50% 사이를 지정한다. 훈련이 끝난 뒤에는 각 입력의 연결 가중치에 보존 확률 (1-p)를 곱해야 한다.(드롭 아웃되어 원래보다 더 많은 뉴런과 연결되었기 때문)

keras.layers.Dropout 층에 전부 구현되어있다.

In [ ]:
model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28,28]),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(300,activation='elu',kernel_initializer='he_normal'),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(100,activation='elu',kernel_initializer='he_normal'),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(10,activation='softmax')
])

# 모델이 과대적합되었다면 드롭아웃 비율을 늘릴 수 있다.
# 모델이 과소적합되었다면 드롭아웃 비율을 낮추는 것이 좋다.
# 드롭아웃은 훈련하는 동안에만 활성화되므로 훈련이 끝난 후 드롭아웃을 빼고 훈련 손실을 평가해야 한다.

# 요약
대체로 잘 작동하는 기준(절대적인 것은 아님)

커널 초기화 : He 초기화<br>
활성화 함수 : ELU<br>
정규화 : 얕은 신경망의 경우 필요 없음, 깊은 신경망이면 배치 정규화<br>
규제 : 조기 종료(필요하면 l2 규제 추가)<br>
옵티마이저 : 모멘텀 최적화(or RMSProp이나 Ndam)
학습률 스케줄 : 1사이클
<br>
<br>
<br>
네트워크가 완전 연결층을 쌓는 단순한 모델이라면 자기 정규화 사용 가능

커널 초기화 : 르쿤 초기화<br>
활성화 함수 : SELU<br>
정규화 : 없음(자기 정규화)<br>
규제 : 필요하면 알파 드롭아웃<br>
옵티마이저 : 모멘텀 최적화(or RMSProp이나 Ndam)
학습률 스케줄 : 1사이클